In [6]:
import pandas as pd
import re
import numpy as np
import nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.cross_validation import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.metrics import classification_report
from nltk.stem.porter import *
from nltk.corpus import stopwords

train= pd.read_csv("train.csv", sep=",")
test = pd.read_csv("test.csv", sep=",")


stopwords = set(stopwords.words('english'))
happy = set([
    u':-)', u':)', u';)', u':o)', u':]', u':3', u':c)', u':>', u'=]', u'8)', u'=)', u':}',
    u':^)', u':-D', u':D', u'8-D', u'8D', u'x-D', u'xD', u'X-D', u'XD', u'=-D', u'=D',
    u'=-3', u'=3', u':-))', u":'-)", u":')", u':*', u':^*', u'>:P', u':-P', u':P', u'X-P',
    u'x-p', u'xp', u'XP', u':-p', u':p', u'=p', u':-b', u':b', u'>:)', u'>;)', u'>:-)',
    u'<3', u'8-)'
    ])

sad = set([
    u':L', u':-/', u'>:/', u':S', u'>:[', u':@', u':-(', u':[', u':-||', u'=L', u':<',
    u':-[', u':-<', u'=\\', u'=/', u'>:(', u':(', u'>.<', u":'-(", u":'(", u':\\', u':-c',
    u':c', u':{', u'>:\\', u';('
    ])

def tokenize( raw_tweet ):
    words = raw_tweet.lower().split()   
    words = [w for w in words if not w in stopwords]
    for one_sentence in words:    
        if one_sentence in sad:
            words[words.index(one_sentence)] = 'bad fuck sad'
        elif one_sentence in happy:
            words[words.index(one_sentence)] = 'happi great happy'
        elif re.match(r'^https?:\/\/.*[\r\n]*',one_sentence):
            words[words.index(one_sentence)] = re.sub(r'^https?:\/\/.*[\r\n]*', '', one_sentence, flags=re.MULTILINE)
        elif re.match(r'(?:@[\w_]+)',one_sentence):
            words[words.index(one_sentence)] = re.sub(r'(?:@[\w_]+)', '', one_sentence)
        elif re.match(r'\d+', one_sentence):
            words[words.index(one_sentence)]  = ''
        else:
            words[words.index(one_sentence)] = re.sub(r'[^\w\s]','',one_sentence)
        
    stemmer = PorterStemmer()
    words = [stemmer.stem(word) for word in words]

    return( " ".join( words ))

clean_train_tweets = []

for index, data in train.iterrows():
    clean_train_tweets.append(tokenize(data['Tweet']))

    
vectorizer = CountVectorizer(analyzer = "word", tokenizer = None,    
                             preprocessor = None,stop_words = None,   
                             max_features = 5000,min_df = 10,max_df = 0.80) 
    
 
train_features = vectorizer.fit_transform(clean_train_tweets)
train_features = train_features.toarray()


clean_test_tweets = [] 

for index, data in test.iterrows():
    clean_test_tweets.append(tokenize(data['Category']))



test_features = vectorizer.transform(clean_test_tweets)
test_features = test_features.toarray()



forest = RandomForestClassifier(n_estimators = 100) 
forest = forest.fit( train_features, train["Category"] )
forest = forest.predict(test_features)


output = pd.DataFrame(data={"Id": test["Id"], "Category": forest})
output = output.sort_index(axis=1, ascending=False)

output.to_csv( "submission.csv", index=False )
